In [7]:
import itertools
import numpy as np
from functools import partial
from tot.models import gpt
import os, re
from tot.pattern_match import check_and_fix_last_line
import time
import requests
from tot.methods.bfs import *
from tot.tasks import get_task
import argparse



def parse_args():
    args = argparse.ArgumentParser()
    args.add_argument("--temperature", type=float, default=0.9)
    args.add_argument("--task", type=str, required=True, choices=["game24", "text", "crosswords"], default="game24")
    args.add_argument("--task_start_index", type=int, default=900)
    args.add_argument("--task_end_index", type=int, default=901)
    args.add_argument("--prompt_sample", type=str, choices=["standard", "cot"], default="standard")
    args.add_argument("--method_generate", type=str, choices=["sample", "propose"], default="propose")
    args.add_argument("--method_evaluate", type=str, choices=["value", "vote"], default="value")
    args.add_argument("--method_select", type=str, choices=["sample", "greedy"], default="greedy")
    args.add_argument("--n_evaluate_sample", type=int, default=3)
    args.add_argument("--n_select_sample", type=int, default=5)
    args.add_argument("--check_format", action='store_true')
    args.add_argument("--eval_rule", action='store_true')
    args.add_argument("--remotebackend", type=str, default="meta-llama-3.1-8b-instruct@q4_k_m")
    args.add_argument("--localbackend", type=str, default="bartowski/Phi-3-medium-128k-instruct-GGUF")
    return args.parse_args([
        "--task", "game24",
        "--task_start_index", "900",
        "--task_end_index", "901",
        "--method_generate", "propose",
        "--method_evaluate", "value",
        "--method_select", "greedy",
        "--n_evaluate_sample", "3",
        "--n_select_sample", "5",
        "--check_format",
        "--eval_rule", 
    ])


args = parse_args()


solve_data = solve_by_one_round(
    args,
    task=get_task(args.task),
    current_task= "4 5 6 10",
    step=0,
    api_base="http://127.0.0.1:11451/v1",
    api_key="lm-studio",
    model="bartowski/Phi-3-medium-128k-instruct-GGUF",
    client_name="local",
    to_print=True,
)


functools.partial(<function gpt at 0x000001940B680F70>, model='bartowski/Phi-3-medium-128k-instruct-GGUF', temperature=0.9, api_base='http://127.0.0.1:11451/v1')
-- new_ys --: ('4 * 5 = 20 (left: 6 10 20)\n', '4 + 5 = 9 (left: 6 9 10)\n', '5 * 6 = 30 (left: 4 10 30)\n', '10 - 4 = 6 (left: 5 6 6)\n', '4 + 6 = 10 (left: 5 10 10)\n', '5 * 10 = 50 (left: 4 6 50)\n', '5 + 6 = 11 (left: 4 10 11)\n')
-- sol values --: (0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
-- choices --: ['4 * 5 = 20 (left: 6 10 20)\n', '4 + 5 = 9 (left: 6 9 10)\n', '5 * 6 = 30 (left: 4 10 30)\n', '10 - 4 = 6 (left: 5 6 6)\n', '4 + 6 = 10 (left: 5 10 10)\n']



In [8]:
a = {
        "client_name": "local_client",
        "api_base": "http://127.0.0.1:11451/v1",
        "api_key": "lm-studio",
        "model": "meta-llama-3.1-8b-instruct@q4_k_m",
    }


SyntaxError: invalid syntax (3116662629.py, line 8)

In [10]:
def assign_task(model_list, ys, **kwargs):
    """
    input: ys: list of str which are the selected output candidates
    **kwargs: a dict of the dicts of api_base, model, api_key, client_name
    output: a list, like that
    [[task1, task2, task3], [task4, task5, task6], [task7, task8, task9]]
    tasks are assigned to different clients with the same index in the model_list
    """
    # 1:1 assignment
    task_list = []
    for i in range(len(model_list)):
        task_list.append([])
    for i in range(len(ys)):
        task_list[i % len(model_list)].append(ys[i])
    return task_list

model_list = [
    {
        "client_name": "local_client",
        "api_base": "http://127.0.0.1:11451/v1",
        "api_key": "lm-studio",
        "model": "meta-llama-3.1-8b-instruct@q4_k_m",
    },
    {
        "client_name": "remote_client",
        "api_base": "http://158.132.255.40:1234/v1",
        "api_key": "lm-studio",
        "model": "qwen2.5-32b-instruct",
    },
]

ys = ["4 5 6 10", "1 1 3 8", "5 5 5 5", "1 2 3 4", "1 1 1 1"]
assign_task(model_list, ys)

[['4 5 6 10', '5 5 5 5', '1 1 1 1'], ['1 1 3 8', '1 2 3 4']]